In [20]:
pip install tensorflow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 766.7 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.3/236.3 MB 8.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 13.1 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.8 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 25.8/25.8 MB 12.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.4/394.4 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 15.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.9/284.9 kB 15.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install torch sentence-transformers faiss


ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from typing import List
import matplotlib.pyplot as plt
import re
import torch
from transformers import AutoTokenizer, AutoModel
import faiss
import numpy as np


In [3]:
main_df = pd.read_json('marketing_sample_for_amazon_com-amazon_fashion_products__20200201_20200430__30k_data.ldjson', lines = True)
main_df.head()

,uniq_id,crawl_timestamp,asin,product_url,product_name,image_urls__small,medium,large,browsenode,brand,...,colour,no__of_reviews,seller_name,seller_id,left_in_stock,no__of_offers,no__of_sellers,technical_details__k_v_pairs,formats___editions,name_of_author_for_books
0,26d41bdc1495de290bc8e6062d927729,2020-02-07 05:11:36 +0000,B07STS2W9T,https://www.amazon.in/Facon-Kalamkari-Handbloc...,LA' Facon Cotton Kalamkari Handblock Saree Blo...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,1.968255e+09,LA' Facon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,410c62298852e68f34c35560f2311e5a,2020-02-07 08:45:56 +0000,B07N6TD2WL,https://www.amazon.in/Sf-Jeans-Pantaloons-T-Sh...,Sf Jeans By Pantaloons Men's Plain Slim fit T-...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,1.968123e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,52e31bb31680b0ec73de0d781a23cc0a,2020-02-06 11:09:38 +0000,B07WJ6WPN1,https://www.amazon.in/LOVISTA-Traditional-Prin...,LOVISTA Cotton Gota Patti Tassel Traditional P...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,1.968255e+09,LOVISTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,25798d6dc43239c118452d1bee0fb088,2020-02-07 08:32:45 +0000,B07PYSF4WZ,https://www.amazon.in/People-Printed-Regular-T...,People Men's Printed Regular fit T-Shirt,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,1.968123e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ad8a5a196d515ef09dfdaf082bdc37c4,2020-02-06 14:27:48 +0000,B082KXNM7X,https://www.amazon.in/Monte-Carlo-Cotton-Colla...,Monte Carlo Grey Solid Cotton Blend Polo Colla...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,https://images-na.ssl-images-amazon.com/images...,1.968070e+09,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
main_df.info

<bound method DataFrame.info of                                 uniq_id            crawl_timestamp  \
0      26d41bdc1495de290bc8e6062d927729  2020-02-07 05:11:36 +0000   
1      410c62298852e68f34c35560f2311e5a  2020-02-07 08:45:56 +0000   
2      52e31bb31680b0ec73de0d781a23cc0a  2020-02-06 11:09:38 +0000   
3      25798d6dc43239c118452d1bee0fb088  2020-02-07 08:32:45 +0000   
4      ad8a5a196d515ef09dfdaf082bdc37c4  2020-02-06 14:27:48 +0000   
...                                 ...                        ...   
29995  976b6bda7076509778da69eb3fe0f59c  2020-02-06 23:53:43 +0000   
29996  3006a520f71804b055d92216f5dc946d  2020-02-06 20:32:09 +0000   
29997  45ea7c463997f4cd91851617edbfee32  2020-02-06 19:55:44 +0000   
29998  2ecada524df6ff8c2d0c53a249cfcddc  2020-02-06 05:08:00 +0000   
29999  a4e77b9b1addb68bdfc4b178ac27e7c2  2020-02-06 23:32:41 +0000   

             asin                                        product_url  \
0      B07STS2W9T  https://www.amazon.in/Facon-Kalamkar

In [5]:
main_df.columns

Index(['uniq_id', 'crawl_timestamp', 'asin', 'product_url', 'product_name',
       'image_urls__small', 'medium', 'large', 'browsenode', 'brand',
       'sales_price', 'weight', 'rating', 'sales_rank_in_parent_category',
       'sales_rank_in_child_category', 'delivery_type', 'meta_keywords',
       'amazon_prime__y_or_n', 'parent___child_category__all',
       'best_seller_tag__y_or_n', 'other_items_customers_buy',
       'product_details__k_v_pairs', 'discount_percentage', 'colour',
       'no__of_reviews', 'seller_name', 'seller_id', 'left_in_stock',
       'no__of_offers', 'no__of_sellers', 'technical_details__k_v_pairs',
       'formats___editions', 'name_of_author_for_books'],
      dtype='object')

In [6]:
df = pd.read_csv('Final_cleaned_dataset.csv')
df.head()

,uniq_id,brand,sales_price,weight,rating
0,26d41bdc1495de290bc8e6062d927729,la' facon,200.0,321.248428,5.0
1,410c62298852e68f34c35560f2311e5a,sf jeans,265.0,539.472488,3.6
2,52e31bb31680b0ec73de0d781a23cc0a,lovista,660.0,264.726016,3.5
3,25798d6dc43239c118452d1bee0fb088,people,195.0,264.726016,3.0
4,ad8a5a196d515ef09dfdaf082bdc37c4,monte carlo,1914.0,321.248428,5.0


In [7]:
df.info

<bound method DataFrame.info of                                 uniq_id                       brand  \
0      26d41bdc1495de290bc8e6062d927729                   la' facon   
1      410c62298852e68f34c35560f2311e5a                    sf jeans   
2      52e31bb31680b0ec73de0d781a23cc0a                     lovista   
3      25798d6dc43239c118452d1bee0fb088                      people   
4      ad8a5a196d515ef09dfdaf082bdc37c4                 monte carlo   
...                                 ...                         ...   
29995  976b6bda7076509778da69eb3fe0f59c              indian virasat   
29996  3006a520f71804b055d92216f5dc946d  urban ranger by pantaloons   
29997  45ea7c463997f4cd91851617edbfee32               peter england   
29998  2ecada524df6ff8c2d0c53a249cfcddc                  pinky pari   
29999  a4e77b9b1addb68bdfc4b178ac27e7c2                       gutsy   

       sales_price      weight  rating  
0            200.0  321.248428     5.0  
1            265.0  539.472488   

In [8]:
df.columns

Index(['uniq_id', 'brand', 'sales_price', 'weight', 'rating'], dtype='object')

In [9]:
df = pd.read_csv('Final_cleaned_dataset.csv')

# Merge the two DataFrames to create final_df
final_df = pd.merge(df, main_df[['uniq_id', 'product_name']], on='uniq_id', how='left')

print(final_df.head())

                            uniq_id        brand  sales_price      weight  \
0  26d41bdc1495de290bc8e6062d927729    la' facon        200.0  321.248428   
1  410c62298852e68f34c35560f2311e5a     sf jeans        265.0  539.472488   
2  52e31bb31680b0ec73de0d781a23cc0a      lovista        660.0  264.726016   
3  25798d6dc43239c118452d1bee0fb088       people        195.0  264.726016   
4  ad8a5a196d515ef09dfdaf082bdc37c4  monte carlo       1914.0  321.248428   

   rating                                       product_name  
0     5.0  LA' Facon Cotton Kalamkari Handblock Saree Blo...  
1     3.6  Sf Jeans By Pantaloons Men's Plain Slim fit T-...  
2     3.5  LOVISTA Cotton Gota Patti Tassel Traditional P...  
3     3.0           People Men's Printed Regular fit T-Shirt  
4     5.0  Monte Carlo Grey Solid Cotton Blend Polo Colla...  


# **Text Pre-Processing**

In [11]:
!pip install faiss-cpu


In [12]:
import pandas as pd
import re
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer, PorterStemmer
from transformers import BertTokenizer, BertModel
import torch
import faiss
import numpy as np

In [13]:
# Download necessary NLTK data
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mriduljhawar/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/mriduljhawar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/mriduljhawar/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [14]:
final_df 


,uniq_id,brand,sales_price,weight,rating,product_name,similarity_score
0,26d41bdc1495de290bc8e6062d927729,la' facon,200.0,321.248428,5.0,LA' Facon Cotton Kalamkari Handblock Saree Blo...,1.000000
1,410c62298852e68f34c35560f2311e5a,sf jeans,265.0,539.472488,3.6,Sf Jeans By Pantaloons Men's Plain Slim fit T-...,-0.054999
2,52e31bb31680b0ec73de0d781a23cc0a,lovista,660.0,264.726016,3.5,LOVISTA Cotton Gota Patti Tassel Traditional P...,-0.276467
3,25798d6dc43239c118452d1bee0fb088,people,195.0,264.726016,3.0,People Men's Printed Regular fit T-Shirt,-0.296251
4,ad8a5a196d515ef09dfdaf082bdc37c4,monte carlo,1914.0,321.248428,5.0,Monte Carlo Grey Solid Cotton Blend Polo Colla...,0.149517
...,...,...,...,...,...,...,...
29995,976b6bda7076509778da69eb3fe0f59c,indian virasat,899.0,321.248428,5.0,Indian Virasat Women's Rayon Anarkali Kurta,0.501674
29996,3006a520f71804b055d92216f5dc946d,urban ranger by pantaloons,204.0,264.726016,3.0,Urban Ranger by Pantaloons Men's Slim fit T-Shirt,-0.299367
29997,45ea7c463997f4cd91851617edbfee32,peter england,399.0,539.472488,4.0,Peter England Men's Regular fit T-Shirt,0.166780
29998,2ecada524df6ff8c2d0c53a249cfcddc,pinky pari,799.0,539.472488,4.0,PINKY PARI Women's Embroidered Short Denim Str...,-0.000749


In [15]:
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Preprocess text function with lemmatization and stemming
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    words = nltk.word_tokenize(text)
    words = [lemmatizer.lemmatize(word) for word in words]
    words = [stemmer.stem(word) for word in words]
    return ' '.join(words)

In [16]:
!pip install langchain


In [ ]:

# Preprocess text function
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

# Initialize model and tokenizer
model_ckpt = "sentence-transformers/multi-qa-mpnet-base-dot-v1"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

# CLS pooling function
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

# Function to get embeddings
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    with torch.no_grad():
        model_output = model(**encoded_input)
    return cls_pooling(model_output).numpy()


# Preprocess product names
final_df['preprocessed_product_name'] = final_df['product_name'].apply(preprocess_text)

# Generate embeddings
embeddings = get_embeddings(final_df['preprocessed_product_name'].tolist())

# Convert embeddings to numpy arrays
embeddings_np = embeddings.astype('float32')

# Add FAISS index to the embeddings
index = faiss.IndexFlatL2(embeddings_np.shape[1])
index.add(embeddings_np)

# Function to find similar products based on product ID
def find_similar_products(product_id, k=5):
    idx = final_df[final_df['uniq_id'] == product_id].index[0]
    query_vector = embeddings_np[idx].reshape(1, -1)
    D, I = index.search(query_vector, k)
    return final_df.iloc[I[0]]

# Example: Find similar products for a given product ID
similar_products = find_similar_products('4188c32471f46876ed222dfcae360791', k=5)
print(similar_products[['uniq_id', 'product_name']])
